In [2]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [3]:
import ipywidgets as widgets
from IPython.display import display

### import

In [4]:
import paramiko
from scp import SCPClient
import os
import pandas as pd
import re
import time
import subprocess


### CSVの配列Sar⇨meG

In [3]:
#csvに含まれる配列にSaRがあったらmeGに変えるコード
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# テキストボックスの作成
array_input = widgets.Text(
    value='',
    placeholder='表のディレクトリパスを入力してください',
    description='パス:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 実行ボタンが押されたときに呼び出される関数
def on_button_click(b):
    output_area.clear_output()  # 出力エリアをクリア
    with output_area:
        try:
            file_path = array_input.value  # テキストボックスの入力を取得
            
            # ファイルが存在するかチェック
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"File not found: {file_path}")
            
            # CSVファイルの読み込み
            df = pd.read_csv(file_path)
            
            # "SAR" を "meG" に置き換え
            if "SAR" in df['Sequence'].values:
                df.loc[df['Sequence'] == "SAR", 'Sequence'] = "meG"
            
            # CSVファイルの保存
            df.to_csv(file_path, index=False)
            print(f"Updated CSV file saved: {file_path}")
        
        except FileNotFoundError as e:
            print(f"File not found: {e}")
        except pd.errors.EmptyDataError:
            print("The CSV file is empty.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

# ボタンがクリックされたときに関数を呼び出す
run_button.on_click(on_button_click)


Text(value='', description='パス:', placeholder='表のディレクトリパスを入力してください')

Button(button_style='success', description='実行', style=ButtonStyle())

Output()

### csvで実行

In [5]:
import pandas as pd
import ast
# テキストボックスの作成
array_input = widgets.Text(
    value='',
    placeholder='表のディレクトリパスを入力してください',
    description='パス:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 実行ボタンが押されたときに呼び出される関数
def on_button_click(b):
    global value  # 必要であればグローバル変数として宣言
    global data
    global input_list
    with output_area:
        output_area.clear_output()
        file_path = array_input.value  # テキストボックスから現在の入力を取得
        #value = file_path
        data = pd.read_csv(file_path)  
        # アミノ酸配列のみを表から取得
        acid_list = data[['CycPeptMPDB_ID','Sequence']]

        print(f"Sequence の最初の要素: {acid_list.iloc[0]}")
        print(f"Sequence の最初の要素の型: {type(acid_list.iloc[0])}")
        # 必要な変数をここで定義
        segid = "PROA"
        rename_str = "rename_str_example"
        delete_str = "delete_str_example"
        cycpep = "cycpep_example"
        kind = ['G','A','V','L','I','C','M','S','T','D','E','N','Q','R',
                'K','H','F','Y','W','P','dA','dV','dL','dI','dC','dM',
                'dS','dT','dD','dE','dN','dQ','dR','dK','dH','dF','dY','dW','dP',
               'Abu','Sar','Me_Bmt(E)','meL','meV','meA','meI','meM','meS','meT',
                'meD','meN','meQ','meK','meH','meF','meY','meW']
        #アミノ酸配列がリストに含まれていなかったら除外するコード
        for row in acid_list.itertuples(index=False):
            seq_list = ast.literal_eval(row.Sequence)
            seq_id = row.CycPeptMPDB_ID
            if all(acid in kind for acid in seq_list):
                print(f"Processing ID {seq_id} with sequence: {seq_list}")
                input_list = seq_list
                value = seq_id
                try:
                    copy_seed_work(value)
                except NameError:
                    print('Error: copy_seed_work 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in copy_seed_work: {e}')
                try:
                    copy_seed_pdb(value, input_list)
                except NameError:
                    print('Error: copy_seed_pdb 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in copy_seed_pdb: {e}')

                try:
                    rename_pdb_atom(input_list)
                except NameError:
                    print('Error: rename_pdb_atom 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in rename_pdb_atom: {e}')

                try:
                    make_rename_tcl(input_list, value)
                except NameError:
                    print('Error: make_rename_tcl 関数が定義されていません。')
                except Exception as e:
                    print(f'An error occurred in make_rename_tcl: {e}')
                
                try:
                    seed_charmm(input_list)
                except NameError:
                    print("Error: seed_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in seed_charmm: {e}")

                try:
                    in_charminp(value)
                except NameError:
                    print("Error: in_charminp 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_charminp: {e}")

                try:
                    gbsw_in_charmm(input_list)
                except NameError:
                    print("Error: gbsw_in_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_in_charmm: {e}")
                
                try:
                    gbsw_sur_charmm(input_list)
                except NameError:
                    print("Error: gbsw_sur_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_sur_charmm: {e}")

                try:
                    gbsw_out_charmm(input_list)
                except NameError:
                    print("Error: gbsw_out_charmm 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in gbsw_out_charmm: {e}")

                try:
                    in_gbsw_in(value)
                except NameError:
                    print("Error: in_gbsw_in 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_in: {e}")

                try:
                    in_gbsw_sur(value)
                except NameError:
                    print("Error: in_gbsw_sur 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_sur: {e}")

                try:
                    in_gbsw_out(value)
                except NameError:
                    print("Error: in_gbsw_out 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_gbsw_out: {e}")
        
                try:
                    connect_server()
                except NameError:
                    print("Error: connect_server 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in connect_server: {e}")

                try:
                    send_dir(value)
                except NameError:
                    print("Error: send_dir 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in send_dir: {e}")
        
                try:
                    run_sh_gbsw(value)
                except NameError:
                    print("Error: run_sh_gbsw 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in run_sh_gbsw: {e}")

                try:
                    run_sh_hbond(value)
                except NameError:
                    print("Error: run_sh_gbsw 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in run_sh_hbond: {e}")
                try:
                    run_sh_surf(value)
                except NameError:
                    print("Error: run_sh_surf 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in run_sh_surf: {e}")
                    
                try:
                    return_copy_work(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in return_copy_work: {e}")
                    
                try:
                    summarize_solv_free_ene(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in summarize_solv_free_ene: {e}")
                    
                try:
                    summarize_van_der_ener(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in summarize_van_der_ener: {e}")
                    
                try:
                    summarize_coul_ener(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in summarize_coul_ener: {e}")
    
                try:
                    calculate_column_average(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in calculate_column_average: {e}")
                
                try:
                    calculate_hbond_average(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in calculate_hbond_average: {e}")

                try:
                    calculate_surf_average(value)
                except NameError:
                    print("Error: calculate_surf_average 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in calculate_surf_average: {e}")
    
                try:
                    save_results(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in save_results: {e}")
    
                try:
                    update_csv_in(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in update_csv_in: {e}")
                
                try:
                    update_csv_sur(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in update_csv_sur: {e}")
                
                try:
                    update_csv_out(value)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in update_csv_out: {e}")
                try:
                    in_csv(file_path)
                except NameError:
                    print("Error: return_copy_work 関数が定義されていません。")
                except Exception as e:
                    print(f"An error occurred in in_csv: {e}")
                


        

                
# ボタンがクリックされたときに実行されるように設定
run_button.on_click(on_button_click)

Text(value='', description='パス:', placeholder='表のディレクトリパスを入力してください')

Button(button_style='success', description='実行', style=ButtonStyle())

Output()

### 直接入力で実行

In [6]:
import subprocess
import os
import paramiko
import ipywidgets as widgets
from IPython.display import display

# テキストボックスの作成
array_input = widgets.Text(
    value='',
    placeholder='配列を入力してください',
    description='配列:',
)

# ボタンの作成
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 出力エリアの作成
output_area = widgets.Output()

# テキストボックスとボタン、出力エリアの表示
display(array_input, run_button, output_area)

# 実行ボタンが押されたときに呼び出される関数
def on_button_click(b):
    global input_list  # 必要であればグローバル変数として宣言
    global value
    with output_area:
        output_area.clear_output()
        input_text = array_input.value  # テキストボックスから現在の入力を取得
        value = input_text
        input_list = input_text.split(',')  # カンマで分割してリスト化
        print(f'入力された値: {value}')
        print(f'リスト化：{input_list}')
        
        # 必要な変数をここで定義
        segid = "PROA"
        rename_str = "rename_str_example"
        delete_str = "delete_str_example"
        cycpep = "cycpep_example"
        try:
            copy_seed_work(value)
        except NameError:
            print('Error: copy_seed_work 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in copy_seed_work: {e}')
        try:
            copy_seed_pdb(value, input_list)
        except NameError:
            print('Error: copy_seed_pdb 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in copy_seed_pdb: {e}')

        try:
            rename_pdb_atom(input_list)
        except NameError:
            print('Error: rename_pdb_atom 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in rename_pdb_atom: {e}')

        try:
            make_rename_tcl(input_list, value)
        except NameError:
            print('Error: make_rename_tcl 関数が定義されていません。')
        except Exception as e:
            print(f'An error occurred in make_rename_tcl: {e}')
                
        try:
            seed_charmm(input_list)
        except NameError:
            print("Error: seed_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in seed_charmm: {e}")

        try:
            in_charminp(value)
        except NameError:
            print("Error: in_charminp 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_charminp: {e}")

        try:
            gbsw_in_charmm(input_list)
        except NameError:
            print("Error: gbsw_in_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in gbsw_in_charmm: {e}")
                
        try:
            gbsw_sur_charmm(input_list)
        except NameError:
            print("Error: gbsw_sur_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in gbsw_sur_charmm: {e}")

        try:
            gbsw_out_charmm(input_list)
        except NameError:
            print("Error: gbsw_out_charmm 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in gbsw_out_charmm: {e}")

        try:
            in_gbsw_in(value)
        except NameError:
            print("Error: in_gbsw_in 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_gbsw_in: {e}")

        try:
            in_gbsw_sur(value)
        except NameError:
            print("Error: in_gbsw_sur 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_gbsw_sur: {e}")

        try:
            in_gbsw_out(value)
        except NameError:
            print("Error: in_gbsw_out 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in in_gbsw_out: {e}")
        
        try:
            connect_server()
        except NameError:
            print("Error: connect_server 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in connect_server: {e}")

        try:
            send_dir(value)
        except NameError:
            print("Error: send_dir 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in send_dir: {e}")
        
        try:
            run_sh_gbsw(value)
        except NameError:
            print("Error: run_sh_gbsw 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in run_sh_gbsw: {e}")

        try:
            run_sh_hbond(value)
        except NameError:
            print("Error: run_sh_gbsw 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in run_sh_hbond: {e}")
                    
        try:
            return_copy_work(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in return_copy_work: {e}")
                    
        try:
            summarize_solv_free_ene(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in summarize_solv_free_ene: {e}")
                    
        try:
            summarize_van_der_ener(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in summarize_van_der_ener: {e}")
                    
        try:
            summarize_coul_ener(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in summarize_coul_ener: {e}")
    
        try:
            calculate_column_average(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in calculate_column_average: {e}")
                
        try:
            calculate_hbond_average(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in calculate_hbond_average: {e}")

        try:
            calculate_surf_average(value)
        except NameError:
            print("Error: calculate_surf_avera 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in calculate_surf_average: {e}")
    
        try:
            save_results(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in save_results: {e}")
    
        #try:
        #    in_csv(file_path)
        #except NameError:
        #    print("Error: return_copy_work 関数が定義されていません。")
        #except Exception as e:
        #    print(f"An error occurred in in_csv: {e}")
                
        try:
            update_csv_in(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in update_csv_in: {e}")
                
        try:
            update_csv_sur(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in update_csv_sur: {e}")
                
        try:
            update_csv_out(value)
        except NameError:
            print("Error: return_copy_work 関数が定義されていません。")
        except Exception as e:
            print(f"An error occurred in update_csv_out: {e}")


        

                
# ボタンがクリックされたときに実行されるように設定
run_button.on_click(on_button_click)



Text(value='', description='配列:', placeholder='配列を入力してください')

Button(button_style='success', description='実行', style=ButtonStyle())

Output()

### seed_workからコピーする

In [92]:
def copy_seed_work(value):
    # コピー元とコピー先のパスを定義
    source_path = "/Users/teraimao/experiment/work/seed_work"
    destination_path = f"/Users/teraimao/experiment/work/{value}_work"
    
    # ディレクトリが存在するか確認
    import os
    if not os.path.exists(source_path):
        print(f"Error: The source directory '{source_path}' does not exist.")
        return
    
    # コピーコマンドの実行
    #subprocess.run(["cp", "-r", source_path, destination_path], check=True)
    try:
        subprocess.run(["cp", "-r", source_path, destination_path], check=True)
        print(f"Directory copied successfully to '{destination_path}'.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during copying: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

### 配列数に合わせてpdbコピー

In [171]:
#配列を入力すると配列名に合わせてMD計算に使うディレクトリをコピーしてくれるコマンド
def copy_seed_pdb(value, input_list):
    destination_path = f"/Users/teraimao/experiment/work/{value}_work/"
    
    try:
        # 配列の長さに応じてコピーするファイルを選択
        if len(input_list) == 4:
            source_path = "/Users/teraimao/experiment/data/seed_pdb/gl4.pdb"
        elif len(input_list) == 5:
            source_path = "/Users/teraimao/experiment/data/seed_pdb/gl5.pdb"
        elif len(input_list) == 6:
            source_path = "/Users/teraimao/experiment/data/seed_pdb/gl6.pdb"
        elif len(input_list) == 11:
            source_path = "/Users/teraimao/experiment/data/seed_pdb/gl11.pdb"
        else:
            print("Error: 対応するファイルがありません。")
            return
        
        # ファイルが存在するか確認
        if not os.path.exists(source_path):
            print(f"Error: The source file '{source_path}' does not exist.")
            return
        
        # ファイルをコピー
        subprocess.run(["cp", source_path, destination_path], check=True)
        print(f"'{source_path}' was copied successfully to '{destination_path}'.")
        
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while copying the file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


### アミノ酸名変更のbuild.tcl作成

In [94]:
#pdbファイルのアミノ酸名を変えるbuild.tclファイルの作成

def rename_pdb_atom(input_list):
    kind = {'G':'GLY','A':'ALA','V':'VAL','L':'LEU','I':'ILE','C':'CYS',
            'M':'MET','S':'SER','T':'THR','D':'ASP','E':'GLU','N':'ASN','Q':'GLN','R':'ARG',
            'K':'LYS','H':'HIS','F':'PHE','Y':'TYR','W':'TRP','P':'PRO',
            'dA':'DAL','dV':'DVL','dL':'DLU','dI':'DIL','dC':'DCY','dM':'DME',
            'dS':'DSE','dT':'DTH','dD':'DSP','dE':'DGU','dN':'DAS','dQ':'DGL',
            'dR':'DA','dK':'DLS','dH':'DHS','dF':'DPE','dY':'DTY','dW':'DTP','dP':'DPR',
            'ABA':'ABA','SAR':'SAR','BMT':'BMT','meL':'LEU','meV':'VAL','meA':'ALA',
            'meI':'ILE','meM':'MET','meS':'SER','meT':'THR','meD':'ASP','meN':'ASN',
            'meQ':'GLN','meK':'LYS','meH':'HIS','meF':'PHE','meY':'TYR','meW':'TRP'}
    rename_command = []
    for i,atom_name in enumerate(input_list):
        atom = kind.get(atom_name)
        if atom:
            rename = f'[atomselect top "resname GLY and resid {i+1}"] set resname {atom}'
            rename_command.append(rename)
    
    rename_str = "\n".join(rename_command)
    
    if len(input_list) == 4:
        gly = 'gl4'
    elif len(input_list) == 5:
        gly = 'gl5'
    elif len(input_list) == 6:
        gly = 'gly6'
    elif len(input_list) == 11:
        gly = 'gl11'
    #build.tclファイルの作成
    build_script = f"""
mol load pdb ../{gly}.pdb

{rename_str}

[atomselect top "all"] writepdb proa.pdb
exit
    """
    return build_script

### build.tcl実行

In [170]:
vmd_executable = "/Applications/VMD.app/Contents/vmd/vmd_MACOSXARM64"  # VMDのフルパス

def make_rename_tcl(input_list, value):
    # VMDスクリプト内容を生成
    rename_build_tcl = rename_pdb_atom(input_list)
    
    # 作業ディレクトリとスクリプトの出力先を定義
    working_dir = f"/Users/teraimao/experiment/work/{value}_work/rename"
    destination_path = os.path.join(working_dir, "build.tcl")
    
    # 作業ディレクトリが存在しなければ作成
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)
    
    # build.tclファイルを作成
    with open(destination_path, 'w') as file:
        file.write(rename_build_tcl)
    
    # build.tclファイルが正しく作成されたか確認
    if not os.path.exists(destination_path):
        print(f"Error: {destination_path} が存在しません。ファイルパスを確認してください。")
        return
    
    # VMDの実行
    try:
        log_path = os.path.join(working_dir, "log")
        with open(log_path, "w") as log_file:
            subprocess.run(
                [vmd_executable, "-dispdev", "text", "-e", destination_path],
                stdout=log_file, stderr=subprocess.PIPE, check=True, cwd=working_dir
            )
        print(f"VMD executed successfully, and output saved to '{log_path}'.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during VMD execution: {e.stderr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


### charmm.inp作成

In [169]:
#charmm.inpの作成
def seed_charmm(input_list):
    segid = "PROA"
    last = len(input_list)
    cycpep = f'patch CYCPEP PROA 1 PROA {last} setup sort warn'

    # NMEパッチコマンドを生成
    nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'me' in res]
    )
    bmt_nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'BMT' in res]
    )


    # CHARMMスクリプトの生成
    charmm_script = f"""
wrnlev 5
prnlev 5 node 0 

open read card name ../toppar/4.1top_all36_prot.rtf unit 20
read rtf card unit 20
close unit 20

open read card name ../toppar/2_1gbsw_par_all36m_prot.prm unit 10
read parameter card unit 10
close unit 10

open read card name ../rename/proa.pdb unit 21
read sequence pdb unit 21

generate {segid} setup first NONE last NONE
{cycpep}
{nme_patches}
{bmt_nme_patches}
autogen ANGLes DIHEdrals 

rewind unit 21

read coordinate pdb unit 21
close unit 21

!ic fill preserve
ic parameter
ic build
hbuild sele type H* end

bomb -1

open write card name temp.pdb unit 14
write coordinates pdb select all end unit 14

close unit 14

!delete atom select all end


cons fix sele segid {segid} .and. (type N .or. type CA .or. type C .or. type O) end
MINI ABNR NSTEP 10000 NPRINT 100
cons fix clear

open unit 40 write form name "peptide.pdb"
write coor pdb unit 40
close unit 40

open unit 50 write form name "peptide.psf"
write psf card unit 50
close unit 50

system "rm temp.pdb"

stop
    """
    
    return charmm_script


### gbsw_inのcharmm.inp

In [185]:
#gbswのcharmm.inpの作成

def gbsw_in_charmm(input_list,z):
    last = len(input_list)
    cycpep = f'patch CYCPEP PROA 1 PROA {last} setup sort warn'
    
    nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'me' in res]
    )
    bmt_nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'BMT' in res]
    )
    
    gbsw_script = f"""
! Read rtf and parm
open read card name ../toppar/4.1top_all36_prot.rtf unit 10
read rtf card unit 10
close unit 10

open read card name ../toppar/2_1gbsw_par_all36m_prot.prm unit 10
read parameter card unit 10
close unit 10

open read card name ../charmm/peptide.pdb unit 20
read sequ pdb unit 20
close unit 20

generate PROA setup first NONE last NONE
{cycpep}
{nme_patches}
{bmt_nme_patches}
autogen ANGLes DIHEdrals

open unit 25 write form name "peptide_in.psf"
write psf card unit 25
close unit 25

open read card name peptide_in.psf unit 30
read psf card unit 30
close unit 30

!read coor
open read card name ../charmm/peptide.pdb unit 20
read coor pdb unit 20
close unit 20

ic para
hbuild

!read gbsw CMAP parameter
open read card name ../../../data/par_all22_prot_gbsw.inp unit 15
read parameter card unit 15
close unit 15

set temp = 303.15

wrnlev 5
prnlev 0 
stream ../../../data/radii_prot_na.str
stream ../../../data/radius_gbsw.str
stream ../../../data/radius.str
prnlev 5

CONS hmcm Force 100.0 weig refx 0.0 refy 0.0 refz {z} sele segid PROA end

scalar wmain statistics select .not. type H* end
define check select ( .not. type H* ) .and. ( property wmain .eq. 0.0 ) show end
if ?nsel ne 0 stop

GBSW SW 0.3 RMAX 20.0 NANG 38 SGAMMA 0.04 TMEMB 30 MSW 2.5 CONC 0.15 TEMP @temp GBENER

NBOND atom switch cdie vdw vswitch ctonnb 12 ctofnb 14 cutnb 18

open read card name ../toppar/4.1top_all36_prot.rtf unit 10
read rtf card unit 10
close unit 10

open read card name ../toppar/2_1par_all36m_prot.prm unit 10
read parameter card unit 10
close unit 10

!ENERGY

mini abnr nstep 1000 nprint 10 step 0.002 inbfrq 10


energy

coor hbond select segid PROA end

open write unform name gbsw_in.rst unit 26
open write unform name gbsw_in.dcd unit 26

shake BONH para tol 1.0d-07

scalar fbeta set 3.0 select all end
scalar fbeta set 0.0 select type H* end
DYNA LEAP LANGEVIN start NSTEP 1000000 TIMEstp 0.0005 NPRINT 2000 -
     INBFRQ 25 TBATH 303.15 FIRSTT 303.15 FINALt 303.15 TSTRUC 303.15 - 
     NSAVcrd 2000 ISEED 12345 IUNCRD 26

SHAKE OFF

mini abnr nstep 1000 nprint 10

close unit 25


stop
    """
    return gbsw_script

### gbsw_surのcharmm.inp

In [190]:
def gbsw_sur_charmm(input_list):
    last = len(input_list)
    cycpep = f'patch CYCPEP PROA 1 PROA {last} setup sort warn'
    nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'me' in res]
    )
    bmt_nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'BMT' in res]
    )
    
    gbsw_script = f"""
! Read rtf and parm
open read card name ../toppar/4.1top_all36_prot.rtf unit 10
read rtf card unit 10
close unit 10


open read card name ../toppar/2_1gbsw_par_all36m_prot.prm unit 10
read parameter card unit 10
close unit 10

open read card name ../charmm/peptide.pdb unit 20
read sequ pdb unit 20
close unit 20

generate PROA setup first NONE last NONE
{cycpep}
{nme_patches}
{bmt_nme_patches}
autogen ANGLes DIHEdrals

open unit 25 write form name "peptide_sur.psf"
write psf card unit 25
close unit 25

open read card name peptide_sur.psf unit 30
read psf card unit 30
close unit 30

!read coor
open read card name ../charmm/peptide.pdb unit 20
read coor pdb unit 20
close unit 20

ic para
hbuild

!read gbsw CMAP parameter
open read card name ../../../data/par_all22_prot_gbsw.inp unit 15
read parameter card unit 15
close unit 15

set temp = 303.15

wrnlev 5
prnlev 0 
stream ../../../data/radii_prot_na.str
stream ../../../data/radius_gbsw.str
stream ../../../data/radius.str
prnlev 5

CONS hmcm Force 100.0 weig refx 0.0 refy 0.0 refz 14.0 sele segid PROA end

scalar wmain statistics select .not. type H* end
define check select ( .not. type H* ) .and. ( property wmain .eq. 0.0 ) show end
if ?nsel ne 0 stop

GBSW SW 0.3 RMAX 20.0 NANG 38 SGAMMA 0.04 TMEMB 30 MSW 2.5 CONC 0.15 TEMP @temp GBENER

NBOND atom switch cdie vdw vswitch ctonnb 12 ctofnb 14 cutnb 18

open read card name ../toppar/4.1top_all36_prot.rtf unit 10
read rtf card unit 10
close unit 10

open read card name ../toppar/2_1par_all36m_prot.prm unit 10
read parameter card unit 10
close unit 10

!ENERGY

mini abnr nstep 1000 nprint 10 step 0.002 inbfrq 10

energy

coor hbond select segid PROA end

open write unform name gbsw_sur.rst unit 26
open write unform name gbsw_sur.dcd unit 26

shake BONH para tol 1.0d-07

scalar fbeta set 3.0 select all end
scalar fbeta set 0.0 select type H* end
DYNA LEAP LANGEVIN start NSTEP 1000000 TIMEstp 0.0005 NPRINT 2000 -
     INBFRQ 25 TBATH 303.15 FIRSTT 303.15 FINALt 303.15 TSTRUC 303.15 - 
     NSAVcrd 2000 ISEED 12345 IUNCRD 26


SHAKE OFF

mini abnr nstep 1000 nprint 10

close unit 25


stop
    """
    return gbsw_script

### gbsw_outのcharmm.inp

In [189]:
def gbsw_out_charmm(input_list):
    last = len(input_list)
    cycpep = f'patch CYCPEP PROA 1 PROA {last} setup sort warn'
    nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'me' in res]
    )
    bmt_nme_patches = "\n".join(
        [f'patch NME PROA {i+1} setup sort warn' for i, res in enumerate(input_list) if 'BMT' in res]
    )
    
    gbsw_script = f"""

! Read rtf and parm
open read card name ../toppar/4.1top_all36_prot.rtf unit 10
read rtf card unit 10
close unit 10

open read card name ../toppar/2_1gbsw_par_all36m_prot.prm unit 10
read parameter card unit 10
close unit 10

open read card name ../charmm/peptide.pdb unit 20
read sequ pdb unit 20
close unit 20

generate PROA setup first NONE last NONE
{cycpep}
{nme_patches}
{bmt_nme_patches}
autogen ANGLes DIHEdrals

open unit 25 write form name "peptide_out.psf"
write psf card unit 25
close unit 25

open read card name peptide_out.psf unit 30
read psf card unit 30
close unit 30

!read coor
open read card name ../charmm/peptide.pdb unit 20
read coor pdb unit 20
close unit 20

ic para
hbuild

!read gbsw CMAP parameter
open read card name ../../../data/par_all22_prot_gbsw.inp unit 15
read parameter card unit 15
close unit 15

set temp = 303.15

wrnlev 5
prnlev 0 
stream ../../../data/radii_prot_na.str
stream ../../../data/radius_gbsw.str
stream ../../../data/radius.str
prnlev 5

CONS hmcm Force 100.0 weig refx 0.0 refy 0.0 refz 16.0 sele segid PROA end

scalar wmain statistics select .not. type H* end
define check select ( .not. type H* ) .and. ( property wmain .eq. 0.0 ) show end
if ?nsel ne 0 stop

GBSW SW 0.3 RMAX 20.0 NANG 38 SGAMMA 0.04 TMEMB 30 MSW 2.5 CONC 0.15 TEMP @temp GBENER

NBOND atom switch cdie vdw vswitch ctonnb 12 ctofnb 14 cutnb 18

open read card name ../toppar/4.1top_all36_prot.rtf unit 10
read rtf card unit 10
close unit 10

open read card name ../toppar/2_1par_all36m_prot.prm unit 10
read parameter card unit 10
close unit 10

!ENERGY

mini abnr nstep 1000 nprint 10 step 0.002 inbfrq 10

energy

coor hbond select segid PROA end

open write unform name gbsw_out.rst unit 26
open write unform name gbsw_out.dcd unit 26

shake BONH para tol 1.0d-07

scalar fbeta set 3.0 select all end
scalar fbeta set 0.0 select type H* end
DYNA LEAP LANGEVIN start NSTEP 1000000 TIMEstp 0.0005 NPRINT 2000 -
     INBFRQ 25 TBATH 303.15 FIRSTT 303.15 FINALt 303.15 TSTRUC 303.15 - 
     NSAVcrd 2000 ISEED 12345 IUNCRD 26


SHAKE OFF

mini abnr nstep 1000 nprint 10

close unit 25


stop
    """
    return gbsw_script

### charmm.inpをvalue_workへ入れる

In [168]:
#作成したcharmm.inpをコピーディレクトリに入れる関数
def in_charminp(value):
    # CHARMMスクリプトを生成
    charmm = seed_charmm(input_list)
    # ディレクトリとファイルパスを設定
    directory = f"/Users/teraimao/experiment/work/{value}_work/charmm"
    destination_path = f"{directory}/charmm.inp"
    
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # ファイルに書き込む
    with open(destination_path, 'w') as file:
        file.write(charmm)
    
        
    

### gbsw_inのcharmm.inpをvalue_workへ入れる

In [167]:
def in_gbsw_in(value):
    # gbsw_in の作成と保存
    gbsw_in = gbsw_in_charmm(input_list)
    directory = f"/Users/teraimao/experiment/work/{value}_work/gbsw_in"
    destination_path = f"{directory}/charmm.inp"
        
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # ファイルに書き込む
    with open(destination_path, 'w') as file:
        file.write(gbsw_in)
    

### gbsw_surのcharmm.inpをvalue_workへ入れる

In [166]:
def in_gbsw_sur(value):
    # gbsw_in の作成と保存
    gbsw_sur = gbsw_sur_charmm(input_list)
    directory = f"/Users/teraimao/experiment/work/{value}_work/gbsw_sur"
    destination_path = f"{directory}/charmm.inp"
        
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # ファイルに書き込む
    with open(destination_path, 'w') as file:
        file.write(gbsw_sur)

### gbsw_outのcharmm.inp⇨value_work

In [165]:
def in_gbsw_out(value):
    # gbsw_in の作成と保存
    gbsw_out = gbsw_out_charmm(input_list)
    directory = f"/Users/teraimao/experiment/work/{value}_work/gbsw_out"
    destination_path = f"{directory}/charmm.inp"
        
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # ファイルに書き込む
    with open(destination_path, 'w') as file:
        file.write(gbsw_out)

### サーバーに接続

In [164]:
#サーバーに接続する関数
def connect_server():
    hostname = "192.168.11.20"  # ログインノードのホスト名
    username = "teraimao"      # ユーザー名
    password = "10Ap1128"      # パスワード
    compute_node = "frigate03"

    try:
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        print(f"Connecting to {hostname}...")
        client.connect(hostname, username=username, password=password)
        print(f"Connected to {hostname}")

        # **計算ノードに SSH する**
        print(f"Entering compute node: {compute_node} ...")

        shell = client.invoke_shell()  # 対話式シェルを開く
        shell.send(f"ssh {compute_node}\n")
        time.sleep(2)  # 少し待つ（接続プロンプトを待つ）
        
        shell.send(f"{password}\n")  # パスワードを送信
        time.sleep(2)  # 認証待ち

        # 結果を取得
        output = shell.recv(65535).decode()
        print(output)

        # もし "Permission denied" が出ていたらエラー
        if "Permission denied" in output:
            print("Error: SSH login to compute node failed.")
            return None

        print(f"Successfully entered {compute_node}")

        return client

    except Exception as e:
        print(f"SSH connection failed: {e}")
        return None


### 計算ノードに接続する

In [163]:
#計算ノードに接続するコード
def connect_compute_node():
    compute_node = "frigate03"  # 計算ノードのホスト名
    client = connect_server()
    if not client:
        return None
    try:
        # シェルセッションを開く
        session = client.invoke_shell()
        time.sleep(1)  # 少し待機してシェルが準備されるのを待つ

        # 計算ノードに接続
        session.send(f"ssh {compute_node}\n")
        time.sleep(1)  # 接続が確立するのを待つ

        # バッファの読み取り
        output = session.recv(1024).decode('utf-8')
        print(f"Connected to compute node {compute_node}:\n{output}")

        # 継続して操作する場合、セッションを返す
        return session
    except Exception as e:
        print(f"Failed to connect to compute node {compute_node}: {e}")
        return None

# メイン部分
#if __name__ == "__main__":
 #   session = connect_compute_node()
  #  if session:
   #     print("Compute node session established. Ready for commands.")
    #else:
     #   print("Failed to establish a compute node session.")

### コピーディレクトリをサーバーに

In [106]:
#コピーディレクトリをサーバーに送る関数
def send_dir(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return
    
    sftp = client.open_sftp()
    local_path = f"/Users/teraimao/experiment/work/{value}_work/"
    remote_path = f'/home/teraimao/experiment/work/{value}_work'

    try:
        sftp.stat(remote_path)
    except FileNotFoundError:
        sftp.mkdir(remote_path)

    try:
        # ディレクトリとファイルを再帰的にアップロード
        upload_directory(sftp, local_path, remote_path)
    except Exception as e:
        print(f"An error occurred in upload_directory: {e}")

    try:
        run_csh(value)
    except Exception as e:
        print(f"An error occurred in run_csh: {e}")
    finally:
        sftp.close()  # SFTPセッションを閉じる
        client.close()  # SSHクライアントを閉じる


In [107]:
#コピーディレクトリの中身をサーバーに送る関数
def upload_directory(sftp, local_path, remote_path):
    for item in os.listdir(local_path):
        local_item = os.path.join(local_path, item)
        remote_item = os.path.join(remote_path, item)

        if os.path.isdir(local_item):
            try:
                sftp.stat(remote_item)
            except FileNotFoundError:
                sftp.mkdir(remote_item)
            upload_directory(sftp, local_item, remote_item)
        else:
            sftp.put(local_item, remote_item)

In [180]:
#charmmディレクトリのrun.cshを実行するコマンド
def run_csh(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return

    remote_directory = f'/home/teraimao/experiment/work/{value}_work/charmm'
    
    # `Run.csh` の実行前に実行権限を付与
    chmod_command = f'cd {remote_directory} && chmod +x Run.csh'
    run_command = f'cd {remote_directory} && ./Run.csh'

    try:
        # `chmod +x Run.csh` を実行
        stdin, stdout, stderr = client.exec_command(chmod_command)
        chmod_error = stderr.read().decode().strip()
        if chmod_error:
            print(f"Error setting executable permission for Run.csh in {remote_directory}:\n{chmod_error}")
            return  # エラーがある場合は実行を中止

        # `Run.csh` を実行
        stdin, stdout, stderr = client.exec_command(run_command)
        output = stdout.read().decode().strip()
        error = stderr.read().decode().strip()
        
        if output:
            print(f"Output:\n{output}")
        if error:
            print(f"Error:\n{error}")

    except Exception as e:
        print(f"An error occurred while executing the script: {e}")
    finally:
        client.close()

In [181]:
#gbsw_のrun.shを実行するコマンド
def run_sh_gbsw(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return

    peptide_positions = ["in", "sur", "out"]

    try:
        for pos in peptide_positions:
            remote_directory = f'/home/teraimao/experiment/work/{value}_work/gbsw_{pos}'

            # `Run.csh` の実行前に実行権限を付与
            chmod_command = f'cd {remote_directory} && chmod +x Run.csh'
            run_command = f'cd {remote_directory} && ./Run.csh'

            # `chmod +x Run.csh` を実行
            stdin, stdout, stderr = client.exec_command(chmod_command)
            chmod_error = stderr.read().decode().strip()
            if chmod_error:
                print(f"Error setting executable permission for Run.csh in {remote_directory}:\n{chmod_error}")
                continue  # エラーがある場合は次の `pos` へスキップ

            # `Run.csh` の実行
            stdin, stdout, stderr = client.exec_command(run_command)
            output = stdout.read().decode().strip()
            error = stderr.read().decode().strip()
            
            if output:
                print(f"Output from {remote_directory}:\n{output}")
            if error:
                print(f"Error from {remote_directory}:\n{error}")

    except Exception as e:
        print(f"An error occurred while executing the script: {e}")
    finally:
        client.close()



### 分子内水素結合を計算する

In [182]:
#分子内水素結合を計算するコマンド
def run_sh_hbond(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return

    peptide_positions = ["in", "sur", "out"]

    try:
        for pos in peptide_positions:
            remote_directory = f'/home/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{pos}/hbond'

            # `Run.csh` の実行前に実行権限を付与
            chmod_command = f'cd {remote_directory} && chmod +x Run.csh'
            run_command = f'cd {remote_directory} && ./Run.csh'

            # `chmod +x Run.csh` を実行
            stdin, stdout, stderr = client.exec_command(chmod_command)
            chmod_error = stderr.read().decode().strip()
            if chmod_error:
                print(f"Error setting executable permission for Run.csh in {remote_directory}:\n{chmod_error}")
                continue  # エラーがある場合は次の `pos` へスキップ

            # `Run.csh` の実行
            stdin, stdout, stderr = client.exec_command(run_command)
            output = stdout.read().decode().strip()
            error = stderr.read().decode().strip()
            
            if output:
                print(f"Output from {remote_directory}:\n{output}")
            if error:
                print(f"Error from {remote_directory}:\n{error}")

    except Exception as e:
        print(f"An error occurred while executing the script: {e}")
    finally:
        client.close()


### 溶媒露出表面積の計算

In [183]:
#溶媒露出表面積の計算
def run_sh_surf(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return

    peptide_positions = ["in", "sur", "out"]

    try:
        for pos in peptide_positions:
            remote_directory = f'/home/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{pos}/surf_area'

            # `Run.csh` の実行前に実行権限を付与
            chmod_command = f'cd {remote_directory} && chmod +x Run.csh'
            run_command = f'cd {remote_directory} && ./Run.csh'

            # `chmod +x Run.csh` を実行
            stdin, stdout, stderr = client.exec_command(chmod_command)
            chmod_error = stderr.read().decode().strip()
            if chmod_error:
                print(f"Error setting executable permission for Run.csh in {remote_directory}:\n{chmod_error}")
                continue  # エラーがある場合は次の `pos` へスキップ

            # `Run.csh` の実行
            stdin, stdout, stderr = client.exec_command(run_command)
            output = stdout.read().decode().strip()
            error = stderr.read().decode().strip()
            
            if output:
                print(f"Output from {remote_directory}:\n{output}")
            if error:
                print(f"Error from {remote_directory}:\n{error}")

    except Exception as e:
        print(f"An error occurred while executing the script: {e}")
    finally:
        client.close()

In [155]:
# 計算済みディレクトリをMacにコピーする関数
def return_copy_work(value):
    client = connect_server()
    if client is None:
        print("Failed to connect to the server.")
        return

    # リモートとローカルのパスを定義
    remote_path = f"/home/teraimao/experiment/work/{value}_work"
    local_path = f"/Users/teraimao/experiment/work/"

    try:
        with SCPClient(client.get_transport()) as scp:
            print(f"Downloading {remote_path} to {local_path} ...")
            scp.get(remote_path, local_path, recursive=True)  # recursive=Trueでディレクトリ対応
            print("Download complete!")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        client.close()

In [156]:
#後半100psの溶媒和自由エネルギーをまとめるコード
def summarize_solv_free_ene(value):
    peptide_positions = ["in", "sur", "out"]  # 修正: 配列名をわかりやすく変更
    
    try:
        for position in peptide_positions:
            # 動的にawkコマンドを生成
            awk_command = f"""
            awk '/^DYNA>/ {{time = $3; next}} /^DYNA PBEQ>/ {{if (time >= 400 && time <= 500) print time, $5}}' \\
            /Users/teraimao/experiment/work/{value}_work/gbsw_{position}/log > \\
            /Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/solv_free_ener/output.dat
            """
            
            # コマンドを実行
            subprocess.run(awk_command, shell=True, check=True)
            print(f"Command executed successfully for position: {position}")
    
    except subprocess.CalledProcessError as e:
        print(f"An error occurred for position {position}: {e}")

In [114]:
#後半100psのファンデルワールスエネルギーをまとめるコード
def summarize_van_der_ener(value):
    peptide_positions = ["in", "sur", "out"]  # 修正: 配列名をわかりやすく変更
    
    try:
        for position in peptide_positions:
            # 動的にawkコマンドを生成
            awk_command = f"""
            awk '/^DYNA>/ {{time = $3; next}} /^DYNA EXTERN>/ {{if (time >= 400 && time <= 500) print time, $3}}' \\
            /Users/teraimao/experiment/work/{value}_work/gbsw_{position}/log > \\
            /Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/van_der_ener/output.dat
            """
            # コマンドを実行
            subprocess.run(awk_command, shell=True, check=True)
            print(f"Command executed successfully for position: {position}")
    
    except subprocess.CalledProcessError as e:
        print(f"An error occurred for position {position}: {e}")

In [115]:
#後半100psのクーロン相互作用エネルギーをまとめるコード
def summarize_coul_ener(value):
    peptide_positions = ["in", "sur", "out"]  # 修正: 配列名をわかりやすく変更
    
    try:
        for position in peptide_positions:
            # 動的にawkコマンドを生成
            awk_command = f"""
            awk '/^DYNA>/ {{time = $3; next}} /^DYNA EXTERN>/ {{if (time >= 400 && time <= 500) print time, $4}}' \\
            /Users/teraimao/experiment/work/{value}_work/gbsw_{position}/log > \\
            /Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/coul_ener/output.dat
            """
            
            # コマンドを実行
            subprocess.run(awk_command, shell=True, check=True)
            print(f"Command executed successfully for position: {position}")
    
    except subprocess.CalledProcessError as e:
        print(f"An error occurred for position {position}: {e}")

In [157]:
#後半100ps系の全ての平均値を計算しaverageディレクトリに入れていく
def calculate_column_average(value):
    column_index = 1  # 平均を計算する列（2列目）
    time_index = 0    # 時間列（1列目）
    peptide_positions = ["in", "sur", "out"]
    directories = ["solv_free_ener", "van_der_ener", "coul_ener"]

    try:
        for position in peptide_positions:
            for directory in directories:
                dir_path = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/{directory}"

                # ディレクトリ内のファイルをリスト
                if not os.path.isdir(dir_path):
                    print(f"Directory not found: {dir_path}")
                    continue

                data_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]
                if not data_files:
                    print(f"No files found in directory: {dir_path}")
                    continue

                # 最初のファイルを選択（複数ファイルがある場合は要確認）
                file_path = os.path.join(dir_path, data_files[0])
                values = []

                print(f"Reading file: {file_path}")
                try:
                    with open(file_path, 'r') as file:
                        for line in file:
                            if not line.strip():  # 空行をスキップ
                                continue
                            columns = line.split()
                            time = float(columns[time_index])  # 時間列を取得
                            if time >= 400.0:  # 時間が40以上の場合のみ値を取得
                                values.append(float(columns[column_index]))
                                print(f"Valid data: {line.strip()}")
                except FileNotFoundError:
                    print(f"File not found: {file_path}")
                    continue
                except ValueError:
                    print(f"Invalid data format in file: {file_path}")
                    continue

                # 平均を計算
                if values:
                    average = sum(values) / len(values)
                    print(f"Calculated average: {average}")
                else:
                    print(f"No valid values found in {file_path} for averaging.")
                    average = 0.0

                # 保存先ディレクトリを作成
                average_dir_path = os.path.join(dir_path, "average")
                os.makedirs(average_dir_path, exist_ok=True)

                # 結果を保存
                average_file_path = os.path.join(average_dir_path, "result.txt")
                try:
                    with open(average_file_path, 'w') as avg_file:
                        avg_file.write(f"{average:.6f}\n")  # 小数点以下6桁で保存
                        print(f"Average value saved to {average_file_path}")
                except Exception as e:
                    print(f"Failed to write average to {average_file_path}: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")


### 後半100psのhbondの平均

In [158]:
#後半100psのhbondの平均を求めるコード
def calculate_hbond_average(value):
    column_index = 1  # 水素結合数がある列（2列目）
    time_index = 0    # 計算時間がある列（1列目）
    peptide_positions = ["in", "sur", "out"]  # 各位置で処理
    directory = "hbond"  # hbond専用ディレクトリ名

    try:
        for position in peptide_positions:
            dir_path = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/{directory}"

            # ディレクトリ内のファイルをリスト
            if not os.path.isdir(dir_path):
                print(f"Directory not found: {dir_path}")
                continue

            data_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]
            if not data_files:
                print(f"No files found in directory: {dir_path}")
                continue

            # 最初のファイルを選択（複数ファイルがある場合は要確認）
            file_path = os.path.join(dir_path, data_files[0])
            values = []

            print(f"Reading hbond file: {file_path}")
            try:
                with open(file_path, 'r') as file:
                    # ヘッダー行をスキップ
                    next(file)

                    # データ行を読み取る
                    for line in file:
                        if not line.strip():  # 空行をスキップ
                            continue
                        columns = line.split()
                        time = float(columns[time_index])  # 時間列を取得
                        if time >= 400.0:  # 時間が40以上の場合のみ値を取得
                            values.append(float(columns[column_index]))
                            print(f"Valid hbond data: {line.strip()}")
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue
            except ValueError:
                print(f"Invalid data format in file: {file_path}")
                continue

            # 平均を計算
            if values:
                average = sum(values) / len(values)
                print(f"Calculated hbond average: {average}")
            else:
                print(f"No valid hbond values found in {file_path} for averaging.")
                average = 0.0

            # 保存先ディレクトリを作成
            average_dir_path = os.path.join(dir_path, "average")
            os.makedirs(average_dir_path, exist_ok=True)

            # 結果を保存
            average_file_path = os.path.join(average_dir_path, "result.txt")
            try:
                with open(average_file_path, 'w') as avg_file:
                    avg_file.write(f"{average:.6f}\n")  # 小数点以下6桁で保存
                    print(f"Average hbond value saved to {average_file_path}")
            except Exception as e:
                print(f"Failed to write hbond average to {average_file_path}: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")



### 後半100psの溶媒露出表面積の平均を求めるコード

In [159]:
#後半10psの溶媒露出表面積の平均を求めるコード
def calculate_surf_average(value):
    column_indices = [1, 2]  # 2列目、3列目、4列目、5列目のインデックス
    time_index = 0  # 計算時間のインデックス
    peptide_positions = ["in", "sur", "out"]  # 各位置で処理
    directory = "surf_area"  # hbond専用ディレクトリ名

    try:
        for position in peptide_positions:
            dir_path = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/{directory}"

            # ディレクトリの存在確認
            if not os.path.isdir(dir_path):
                print(f"Directory not found: {dir_path}")
                continue

            # ディレクトリ内のファイルをリストアップ
            data_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]
            if not data_files:
                print(f"No files found in directory: {dir_path}")
                continue

            # 最初のファイルを選択（複数ファイルがある場合は要確認）
            file_path = os.path.join(dir_path, data_files[0])
            column_values = {index: [] for index in column_indices}

            with open(file_path, 'r') as file:
                next(file)  # ヘッダーをスキップ
                for line in file:
                    if not line.strip():  # 空行をスキップ
                        continue
                    columns = line.split()
                    time = float(columns[time_index])  # 時間列を取得
                    if time >= 400.0:  # 時間が40以上の場合のみ値を取得
                        for index in column_indices:
                            column_values[index].append(float(columns[index]))

            # 各列の平均を計算
            averages = {
                index: (sum(column_values[index]) / len(column_values[index])) if column_values[index] else 0.0
                for index in column_indices
            }

            # データがなかった場合の処理
            if not any(column_values.values()):
                print(f"No valid hbond values found in {file_path} for averaging.")
                continue

            # "average" ディレクトリを作成
            average_dir_path = os.path.join(dir_path, "average")
            os.makedirs(average_dir_path, exist_ok=True)

            # 結果を保存
            average_file_path = os.path.join(average_dir_path, "result.txt")
            try:
                with open(average_file_path, 'w') as avg_file:
                    for index, avg in averages.items():
                        avg_file.write(f"Column {index+1} average: {avg:.6f}\n")
                    print(f"Averages saved to {average_file_path}")
            except Exception as e:
                print(f"Failed to write averages to {average_file_path}: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [119]:
#平均値を計算する必要のない表面積を解析ディレクトリに入れる
import os
from collections import OrderedDict

def extract_total_from_log(log_file, target_commands):
    """ログファイルから指定されたターゲットコマンドの次の `total =` の値を抽出する"""
    target_areas = OrderedDict((cmd, None) for cmd in target_commands)  # 順番を保証する辞書

    try:
        with open(log_file, 'r') as file:
            lines = file.readlines()
            for i, line in enumerate(lines):
                for key in target_areas.keys():
                    if key in line:  # ターゲットコマンドを検出
                        for j in range(i + 1, len(lines)):
                            if "total  =" in lines[j]:
                                target_areas[key] = lines[j].split()[-1]  # 値を取得
                                break
        return target_areas
    except FileNotFoundError:
        print(f"Log file not found: {log_file}")
        return None
    except Exception as e:
        print(f"Error reading log file: {e}")
        return None

def save_results(value):
    peptide_positions = ["in", "sur", "out"]
    target_commands = [
        "COOR SURF RPROBE 1.4 SELE all END",  # surf_area 用
        "COOR SURF RPROBE 1.4 SELE type O* .or. type N* END",  # polar_surf_area 用
    ]

    for position in peptide_positions:
        # パスの定義
        log_file = f"/Users/teraimao/experiment/work/{value}_work/gbsw_{position}/log"
        dir_polar = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/polar_surf_area/average"
        dir_surf = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/surf_area/average"

        # ログファイルからデータを抽出
        target_areas = extract_total_from_log(log_file, target_commands)
        if not target_areas:
            print(f"Skipping {log_file} due to missing or invalid data.")
            continue

        # 値を保存
        if None not in target_areas.values():  # 全てのターゲットが見つかった場合
            os.makedirs(dir_polar, exist_ok=True)
            os.makedirs(dir_surf, exist_ok=True)

            # 正しい対応関係で保存する
            with open(os.path.join(dir_surf, "result.txt"), 'w') as file_surf:
                file_surf.write(target_areas["COOR SURF RPROBE 1.4 SELE all END"] + "\n")

            with open(os.path.join(dir_polar, "result.txt"), 'w') as file_polar:
                file_polar.write(target_areas["COOR SURF RPROBE 1.4 SELE type O* .or. type N* END"] + "\n")

            print(f"Values saved successfully for {position}:")
            print(f" - {target_areas['COOR SURF RPROBE 1.4 SELE all END']} saved to {os.path.join(dir_surf, 'result.txt')}")
            print(f" - {target_areas['COOR SURF RPROBE 1.4 SELE type O* .or. type N* END']} saved to {os.path.join(dir_polar, 'result.txt')}")
        else:
            print(f"Error: Missing data for some target commands in {log_file}.")


### inの計算結果をcsvファイルに

In [160]:
#inの計算結果をcsvファイルに入れるコード
def update_csv_in(value):
    csv_list = ['coul_ener', 'hbond', 'van_der_ener', 'solv_free_ener']
    csv_experiment_data = ['クーロン相互作用エネルギー', '分子内水素結合数', 'ファンデルワールスエネルギー', '溶媒和自由エネルギー']
    
    surf_map = {
    2: '溶媒露出表面積',
    3: '3D-PSA'
    }

    csv_file = "/Users/teraimao/experiment/data/csv_data/experiment_data_in.csv"
    results = {}

    # **エネルギー関連の値を取得**
    for j, column_name in zip(csv_list, csv_experiment_data):
        result_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_in/{j}/average/result.txt"
        
        if not os.path.exists(result_file):
            print(f"Warning: {result_file} not found. Initializing as 0.0")
            results[column_name] = 0.0
            continue

        try:
            with open(result_file, 'r') as file:
                file_value = file.readline().strip()
                print(f"DEBUG: Read value for {column_name} -> '{file_value}'")
                results[column_name] = float(file_value) if file_value else 0.0
        except ValueError:
            print(f"ERROR: Could not convert {file_value} to float for {column_name}")
            results[column_name] = 0.0

    # **溶媒露出表面積関連の値を取得**
    surf_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_in/surf_area/average/result.txt"
    if os.path.exists(surf_file) and not os.path.isdir(surf_file):
        try:
            with open(surf_file, 'r') as file:
                for line in file:
                    match = re.match(r"Column (\d+) average: ([\d\.]+)", line.strip())
                    if match:
                        col_num = int(match.group(1))
                        avg_value = float(match.group(2))
                        if col_num in surf_map:
                            results[surf_map[col_num]] = avg_value
        except Exception as e:
            print(f"Error reading {surf_file}: {e}")
    else:
        print(f"Warning: {surf_file} not found. Initializing values as 0.")
        for col_num in surf_map:
            results[surf_map[col_num]] = 0.0

    # **CSV ファイルの更新**
    try:
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
        else:
            df = pd.DataFrame(columns=['CycPeptMPDB_ID'] + list(results.keys()))
            print(f"Created a new CSV file: {csv_file}")

        for column_name in results.keys():
            if column_name not in df.columns:
                df[column_name] = None

        # **新しい `CycPeptMPDB_ID` を追加**
        new_row = {'CycPeptMPDB_ID': value, **results}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

        df.to_csv(csv_file, index=False)
        print(f"Updated CSV file saved: {csv_file}")
    except Exception as e:
        print(f"Error updating CSV file: {e}")


In [161]:
#surの計算結果をcsvファイルに入れるコード
def update_csv_sur(value):
    csv_list = ['coul_ener', 'hbond', 'van_der_ener', 'solv_free_ener']
    csv_experiment_data = ['クーロン相互作用エネルギー', '分子内水素結合数', 'ファンデルワールスエネルギー', '溶媒和自由エネルギー']
    
    surf_map = {
    2: '溶媒露出表面積',
    3: '3D-PSA'
    }

    csv_file = "/Users/teraimao/experiment/data/csv_data/experiment_data_sur.csv"
    results = {}

    # **エネルギー関連の値を取得**
    for j, column_name in zip(csv_list, csv_experiment_data):
        result_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_sur/{j}/average/result.txt"
        
        if not os.path.exists(result_file):
            print(f"Warning: {result_file} not found. Initializing as 0.0")
            results[column_name] = 0.0
            continue

        try:
            with open(result_file, 'r') as file:
                file_value = file.readline().strip()
                print(f"DEBUG: Read value for {column_name} -> '{file_value}'")
                results[column_name] = float(file_value) if file_value else 0.0
        except ValueError:
            print(f"ERROR: Could not convert {file_value} to float for {column_name}")
            results[column_name] = 0.0

    # **溶媒露出表面積関連の値を取得**
    surf_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_sur/surf_area/average/result.txt"
    if os.path.exists(surf_file) and not os.path.isdir(surf_file):
        try:
            with open(surf_file, 'r') as file:
                for line in file:
                    match = re.match(r"Column (\d+) average: ([\d\.]+)", line.strip())
                    if match:
                        col_num = int(match.group(1))
                        avg_value = float(match.group(2))
                        if col_num in surf_map:
                            results[surf_map[col_num]] = avg_value
        except Exception as e:
            print(f"Error reading {surf_file}: {e}")
    else:
        print(f"Warning: {surf_file} not found. Initializing values as 0.")
        for col_num in surf_map:
            results[surf_map[col_num]] = 0.0

    # **CSV ファイルの更新**
    try:
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
        else:
            df = pd.DataFrame(columns=['CycPeptMPDB_ID'] + list(results.keys()))
            print(f"Created a new CSV file: {csv_file}")

        for column_name in results.keys():
            if column_name not in df.columns:
                df[column_name] = None

        # **新しい `CycPeptMPDB_ID` を追加**
        new_row = {'CycPeptMPDB_ID': value, **results}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

        df.to_csv(csv_file, index=False)
        print(f"Updated CSV file saved: {csv_file}")
    except Exception as e:
        print(f"Error updating CSV file: {e}")


In [162]:
#outの計算結果をcsvファイルに入れるコード
def update_csv_out(value):
    csv_list = ['coul_ener', 'hbond', 'van_der_ener', 'solv_free_ener']
    csv_experiment_data = ['クーロン相互作用エネルギー', '分子内水素結合数', 'ファンデルワールスエネルギー', '溶媒和自由エネルギー']
    
    surf_map = {
    2: '溶媒露出表面積',
    3: '3D-PSA'
    }

    csv_file = "/Users/teraimao/experiment/data/csv_data/experiment_data_out.csv"
    results = {}

    # **エネルギー関連の値を取得**
    for j, column_name in zip(csv_list, csv_experiment_data):
        result_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_out/{j}/average/result.txt"
        
        if not os.path.exists(result_file):
            print(f"Warning: {result_file} not found. Initializing as 0.0")
            results[column_name] = 0.0
            continue

        try:
            with open(result_file, 'r') as file:
                file_value = file.readline().strip()
                print(f"DEBUG: Read value for {column_name} -> '{file_value}'")
                results[column_name] = float(file_value) if file_value else 0.0
        except ValueError:
            print(f"ERROR: Could not convert {file_value} to float for {column_name}")
            results[column_name] = 0.0

    # **溶媒露出表面積関連の値を取得**
    surf_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_out/surf_area/average/result.txt"
    if os.path.exists(surf_file) and not os.path.isdir(surf_file):
        try:
            with open(surf_file, 'r') as file:
                for line in file:
                    match = re.match(r"Column (\d+) average: ([\d\.]+)", line.strip())
                    if match:
                        col_num = int(match.group(1))
                        avg_value = float(match.group(2))
                        if col_num in surf_map:
                            results[surf_map[col_num]] = avg_value
        except Exception as e:
            print(f"Error reading {surf_file}: {e}")
    else:
        print(f"Warning: {surf_file} not found. Initializing values as 0.")
        for col_num in surf_map:
            results[surf_map[col_num]] = 0.0

    # **CSV ファイルの更新**
    try:
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
        else:
            df = pd.DataFrame(columns=['CycPeptMPDB_ID'] + list(results.keys()))
            print(f"Created a new CSV file: {csv_file}")

        for column_name in results.keys():
            if column_name not in df.columns:
                df[column_name] = None

        # **新しい `CycPeptMPDB_ID` を追加**
        new_row = {'CycPeptMPDB_ID': value, **results}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

        df.to_csv(csv_file, index=False)
        print(f"Updated CSV file saved: {csv_file}")
    except Exception as e:
        print(f"Error updating CSV file: {e}")


### 表面積計算から平均値出すまで

In [139]:
def all():
    #value = ["0_18_35_K,N,S,Y,F","13_14_15_K,N,S,Y,F","16_17_18_K,N,S,Y,F",
          #  "19_20_21_K,N,S,Y,F","1_2_3_K,N,S,Y,F","4_5_6_K,N,S,Y,F",
          #  "9_10_11_K,N,S,Y,F","7_8_12_K,N,S,Y,F"]
    value = ["K,N,S,Y,F"]
    for value in value:
        return_copy_work(value)
        calculate_surf_average(value)
        update_csv_in(value)
        update_csv_sur(value)
        update_csv_out(value)

0_18_35_K,N,S,Y,F
13_14_15_K,N,S,Y,F
16_17_18_K,N,S,Y,F
19_20_21_K,N,S,Y,F
1_2_3_K,N,S,Y,F
4_5_6_K,N,S,Y,F
9_10_11_K,N,S,Y,F
7_8_12_K,N,S,Y,F

In [ ]:
        run_sh_hbond(value)
        run_sh_surf(value)
        return_copy_work(value)
        summarize_solv_free_ene(value)
        summarize_van_der_ener(value)
        summarize_coul_ener(value)
        calculate_column_average(value)
        calculate_hbond_average(value)
        calculate_surf_average(value)
        update_csv_in(value)
        update_csv_in(value)
        update_csv_sur(value)
        update_csv_out(value)

In [140]:
all()

Connecting to 192.168.11.20...
Connected to 192.168.11.20
Entering compute node: frigate03 ...
Web console: https://frigate.local:9090/ or https://192.168.11.20:9090/

Last login: Thu Feb 13 14:39:52 2025 from 192.168.11.144
ssh frigate03
[teraimao@frigate ~]$ ssh frigate03
teraimao@frigate03's password: 
Web console: https://frigate03.local:9090/ or https://192.168.18.53:9090/

Last login: Thu Feb 13 14:39:54 2025 from frigate.local
[teraimao@frigate03 ~]$ 
Successfully entered frigate03
Download complete!
Averages saved to /Users/teraimao/experiment/work/K,N,S,Y,F_work/analysis/ana_gbsw_in/surf_area/average/result.txt
Averages saved to /Users/teraimao/experiment/work/K,N,S,Y,F_work/analysis/ana_gbsw_sur/surf_area/average/result.txt
An unexpected error occurred: could not convert string to float: 'read'
DEBUG: Read value for クーロン相互作用エネルギー -> '-99.248910'
DEBUG: Read value for 分子内水素結合数 -> '2.970297'
DEBUG: Read value for ファンデルワールスエネルギー -> '15.281934'
DEBUG: Read value for 溶媒和自由エネルギー ->